In [ ]:
!pip install openai==0.28

!pip install pandas

In [ ]:
# Import required libraries
import openai
import pandas as pd

In [ ]:
# Set OpenAI API Key
openai.api_key = 'sk-proj-f3lgY38z6x2l9KxM97SoT3BlbkFJJlXXXXXXXXXXXXXXXXX'

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
destination_data = pd.read_csv("/content/drive/MyDrive/TravelPlanAI/Traveldetailsdataset.csv")

In [ ]:
destination_data.head()

,Trip ID,Destination,Start date,End date,Duration (days),Traveler name,Traveler age,Traveler gender,Traveler nationality,Accommodation type,Accommodation cost,Transportation type,Transportation cost
0,1,"London, UK",5/1/2023,5/8/2023,7.0,John Smith,35.0,Male,American,Hotel,1200,Flight,600
1,2,"Phuket, Thailand",6/15/2023,6/20/2023,5.0,Jane Doe,28.0,Female,Canadian,Resort,800,Flight,500
2,3,"Bali, Indonesia",7/1/2023,7/8/2023,7.0,David Lee,45.0,Male,Korean,Villa,1000,Flight,700
3,4,"New York, USA",8/15/2023,8/29/2023,14.0,Sarah Johnson,29.0,Female,British,Hotel,2000,Flight,1000
4,5,"Tokyo, Japan",9/10/2023,9/17/2023,7.0,Kim Nguyen,26.0,Female,Vietnamese,Airbnb,700,Train,200


In [ ]:
# Function to initialize the conversation with a welcome message
def initialize_conversation():
    welcome_message = "Hello! I'm your virtual travel planner. I can help you find the best hotels and plan your itinerary. How can I assist you today?"
    return welcome_message

In [ ]:
# Function to perform moderation check using OpenAI's Moderation API
# Ensures that the user input is appropriate
def moderation_check(user_input):
    response = openai.Moderation.create(input=user_input)
    moderation_decision = response['results'][0]['flagged']
    return moderation_decision

In [ ]:
# Function to confirm the user's intent
# Asks the user to confirm their intent to ensure accuracy
def intent_confirmation_layer(intent):
    confirmation_message = f"Just to confirm, you want to {intent}. Is that correct? (yes/no)"
    return confirmation_message

In [ ]:
# Function to find the best hotels for a given destination
# Filters the dataset for the specified destination and selects the top 5 hotels based on rating
def find_best_hotels(destination):
    # Filter the dataset for the given destination
    hotels = destination_data[destination_data['Destination'].str.contains(destination, case=False)]
    if hotels.empty:
        return [{"error": "No hotels found for the specified destination."}]
    # Select top 5 hotels based on a rating column
    top_hotels = hotels.nlargest(5, 'Rating')[['Hotel Name', 'Rating']]
    return top_hotels.to_dict(orient='records')

In [ ]:
# Function to get the chat model response from OpenAI's API
def get_chat_model_response(user_input):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful travel assistant."},
            {"role": "user", "content": user_input}
        ]
    )
    return response['choices'][0]['message']['content'].strip()

In [ ]:
def main():
    # Print the welcome message to start the conversation
    print(initialize_conversation())

    while True:
        # Take user input from the command line
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            break

        # Simple intent detection for demonstration purposes
        if "hotel" in user_input.lower():
            intent = "find a hotel"
        else:
            intent = "assist with travel planning"

        # Confirm the detected intent with the user
        print(intent_confirmation_layer(intent))
        confirmation = input("You: ").strip().lower()
        if confirmation != 'yes':
            print("Okay, let's try again. How can I assist you?")
            continue

        # Get the response from the chat model
        response = get_chat_model_response(user_input)
        print("Chatbot:", response)

In [ ]:
# Run the main function if the script is executed
if __name__ == '__main__':
    main()

Hello! I'm your virtual travel planner. I can help you find the best hotels and plan your itinerary. How can I assist you today?
Just to confirm, you want to find a hotel. Is that correct? (yes/no)
Chatbot: Sure, here are a few hotels in Paris that I can recommend:

1. Shangri-La Hotel Paris - This 5-star hotel is located in the 16th district of Paris, facing the Eiffel Tower. 

2. Le Bristol Paris - A luxury hotel situated in the heart of Paris, on the famous Rue du Faubourg Saint Honoré. 

3. Hôtel Ritz Paris - Located in 1st district of Paris, Ritz Paris offers luxurious suites, world-class service and a unique location.  

4. Hotel Le A - A chic boutique hotel located close to Champs-Élysées.

5. Hotel Therese - Located on the right bank, offering chic and cozy accommodation with great customer service.

6. Pullman Paris Tour Eiffel - A modern hotel offering stellar views of the Eiffel Tower.

Remember, prices can vary depending on the season and room availability. I suggest checki